In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Importing Data

For the purposes of this project, I have drawn a lot of insight from the Capstone project done here: https://github.com/YaronBlinder/MIMIC-III_readmission/blob/master/Report.pdf 

Based on some of the previous work done, I determined the tables I wanted to download from the Mimic III database. The **admissions** table gives me relevant information such as Subject ID, Admission ID, Admit Time, Discharge Time, Death Time (if died during stay), Marital Status and Insurance provider. From the **patients** table, I can obtain information such Age and Gender. From the **lab events** I was able to determine the numerical values for lab tests based on their test ID and from **D_LABITEMS** I was able to find the lab test IDs for the actual tests that I am interested in (namely Calcium (Total), Albumin, Urea Nitrogen and Magnesium)

Since the **lab events** table was most likely too big to be loaded into memory of the Jupyter Notebook, I parsed out only the columns that I wanted (SUBJECT_ID, HADM_ID, ITEM_ID and VALUENUM) and then took the first 2 million rows of the data set. This was done on command line using the following commands:

*cut -d , -f 2,4-6 LABEVENTS.csv > test.csv*  
*sudo head -n 2000000 test.csv > cleaned_data.csv*

In [ ]:
admissions = pd.read_csv('ADMISSIONS.csv')
labevents = pd.read_csv('cleaned_data.csv')
D_lab = pd.read_csv('D_LABITEMS.csv')
patients = pd.read_csv('PATIENTS.csv')

### Cleaning Up the Data

In the labevents csv file, there were many cases where there was no information in the HADM_ID section. These correspond with outpatients and will not be something that I will be considering. As such, the first step that I took was in removing any NaN values that existed in the HADM_ID column of the labevents dataframe.

Next, since I want to be use age as a statistic, I will calculate the patient's age based on their DOB (unless they have a DOB, in which case I will calculate their age by subtracting the DOB from the DOD). To find the patients that are still alive, since I was uncertain about how the timestamps were recorded, I decided to make the current time one year after the latest date of death and use that as the current time stamp.

To make the dataset easier to visualize, upon the calculating the age, I also removed information that was not relevant to the machine learning process.

In [ ]:
labevents = labevents.dropna()
# removes all rows that have are NaN in the dataframe - in this case, only HADM_ID has NaNs

patients['DOB'] = pd.to_datetime(patients['DOB'])
patients['DOD'] = pd.to_datetime(patients['DOD'])
# converts DOD and DOB to data time objects in the pd dataframe

print((patients['DOD']).max())
currentTime = pd.Timestamp(2212, 6, 10, 10, 10, 10)
# find the current time by adding one year onto the most recent date of death

patients['Age'] = ((patients['DOD'] - patients['DOB']).dt.days)/365
# for the rows that do not have NaN in the DOD column, subtract the DOB from DOD to get the age at their time of death

patients['CurrentTime'] = currentTime
patients['CurrentTime'] = pd.to_datetime(patients['CurrentTime'])

patients['Temp'] = (patients['CurrentTime'] - patients['DOB']).dt.days/365
# creates a temporary column that has the ages of the patients - including all the patients that have died

patients.loc[patients['Age'].isna(), 'Age'] = patients['Temp']
# for all the ages that are nan - i.e. the patients that do not have a DOD value -, replace it with the age calculated

patients = patients[['ROW_ID', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'Age']]

patients.head()

D_lab.head()

labevents.head()

I then further cleaned up, and made the **labevents** table smaller by only keeping the lab tests that I thought were relevant to the machine learning model. I got the *ITEMID* of the tests corresponding to the four categories I listed below.

Calcium (Total) - 50893   
Albumin - 50862   
Urea Nitrogen - 51006   
Magnesium - 50960


In [ ]:
labevents = labevents.loc[(labevents['ITEMID'] == 50893) | (labevents['ITEMID'] == 50862) | 
                          (labevents['ITEMID']==51006) | (labevents['ITEMID'] == 50960)]

labevents.head()

Following this, I combined the **labevents** table with the the **D_LABITEMS** table so that I would be able to better see on the table what each of the lab tests actually was.